In [1]:
import pickle

# with open('page_topic_map.pickle', 'wb') as handle:
#     pickle.dump(page_topic_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('page_topic_map.pickle', 'rb') as handle:
    page_topic_map = pickle.load(handle)


In [2]:

import jsons
reformat = jsons.dump(page_topic_map)

In [7]:

from PyPDF2 import PdfFileWriter, PdfFileReader
import io
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter


def WriteOnPdf (targetpdf, pageTopicsDict):
    if targetpdf == 'State Action Plan_Andhra-pradesh_2012.pdf' or targetpdf == 'State Action Plan_Madhya-Pradesh_2014.pdf':
        return
    existing_pdf = PdfFileReader(open(targetpdf, "rb"))
    output = PdfFileWriter()

    for i in range(existing_pdf.numPages):
        if i in set(pageTopicsDict.keys()):
            packet = io.BytesIO()
            # Create a new PDF with Reportlab
            can = canvas.Canvas(packet, pagesize=letter)
            can.setFillColorRGB(1,0,0) #choose your font colour
            can.setFont('Helvetica', 13)
            can.drawString(5, 830, pageTopicsDict[i])
            can.save()

            # Move to the beginning of the StringIO buffer
            packet.seek(0)
            new_pdf = PdfFileReader(packet)
            page = existing_pdf.getPage(i)
            page.mergePage(new_pdf.getPage(0))# index out of range
            output.addPage(page)    
        
    # Finally, write "output" to a real file
    outputStream = open(targetpdf+"with topics.pdf", "wb")
    output.write(outputStream)
    outputStream.close()

In [8]:
#preprocess that monstrosity of a pickle
# for eachDoc in page_topic_map.keys():
for oneDoc in page_topic_map.keys():
    reformatDict = {}
    for i in reformat[oneDoc]:
        reformatDict[i[0]] = ' '.join(i[1])
    WriteOnPdf(oneDoc, reformatDict)